In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir('..')  # Run this only once

In [67]:
from my_io import Dataset
dataset = Dataset('fraction')

prefix is now tmp/0-0


In [4]:
X = tf.constant(np.array(dataset.data).astype(np.float32))

In [95]:
N = len(dataset.data)
R = 8
M = len(dataset.data[0])

In [96]:
import random
mask = [[random.random() < 0.9 for _ in range(M)] for _ in range(N)]

In [97]:
tf_mask = tf.constant(mask, dtype=tf.float32)

In [98]:
U = tf.Variable(tf.random_normal([N, R]), name='U')
V = tf.Variable(tf.random_normal([R, M]), name='V')
bias = tf.Variable(tf.random_normal([M]), name='bias')

In [170]:
X_est = tf.add(tf.matmul(U, tf.abs(V)), bias)
train_negative_log_likelihood = tf.reduce_sum(tf.multiply(tf_mask, tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=X_est)))
test_negative_log_likelihood = tf.reduce_sum(tf.multiply(1 - tf_mask, tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=X_est)))
loss = train_negative_log_likelihood + 0.1 * tf.reduce_sum(tf.square(U)) + 10 * tf.reduce_sum(tf.abs(V)) + 0.1 * tf.reduce_sum(tf.square(bias))

In [171]:
optimizer = tf.train.AdamOptimizer(0.01)
train_step = optimizer.minimize(loss, var_list=[U, V, bias])
#train_V_step = optimizer.minimize(negative_log_likelihood, var_list=[V])

In [172]:
init_op = tf.global_variables_initializer()

In [173]:
sess = tf.InteractiveSession()
sess.run(init_op)

In [174]:
for i in range(100):
    sess.run(train_step)
#    sess.run(train_V_step)
    if i % 5 == 0:
        print(i, sess.run(test_negative_log_likelihood))

0 1438.86
5 1284.11
10 1158.28
15 1056.25
20 972.484
25 902.743
30 843.999
35 793.907
40 750.765
45 713.294
50 680.562
55 651.94
60 626.988
65 605.349
70 586.698
75 570.715
80 557.076
85 545.462
90 535.574
95 527.044


La bonne nouvelle, c'est qu'on obtient une log-likelihood de -3606 en 20 itérations alors que je viens de check, le package R converge au bout de 210 itérations vers une log-likelihood de -4454.

Reste à savoir si ça généralise bien. (Le package R lui suppose un prior bayésien sur les compétences des gens.)

In [175]:
sess.run(tf.transpose(tf.abs(V)))

array([[  1.22678399e+00,   3.18735242e-01,   5.84169388e-01,
          6.15280926e-01,   8.72528493e-01,   4.21755701e-01,
          6.50577962e-01,   7.08126903e-01],
       [  1.04894519e+00,   4.76301759e-01,   8.70995700e-01,
          7.29382455e-01,   7.97418833e-01,   4.61259931e-01,
          4.77243543e-01,   6.75637424e-01],
       [  9.48334694e-01,   5.89395046e-01,   8.72558355e-01,
          4.55892026e-01,   7.92374969e-01,   4.21434075e-01,
          5.55222154e-01,   9.74047244e-01],
       [  9.28602576e-01,   7.21054256e-01,   4.05707490e-03,
          5.11832237e-01,   2.83757657e-01,   7.08328485e-01,
          9.57066953e-01,   1.63759202e-01],
       [  6.33519769e-01,   3.16899978e-02,   1.52041376e-01,
          3.84081155e-01,   3.71191770e-01,   3.82133245e-01,
          2.06009373e-01,   4.57519382e-01],
       [  6.23870075e-01,   4.61778998e-01,   6.07828200e-01,
          2.79283226e-01,   3.71352792e-01,   1.11932963e-01,
          1.55556574e-01,   4.8